## imports
in this section we import every library and other stuffs we want.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ucimlrepo import fetch_ucirepo 
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.cluster import KMeans


## import data
so I checked our dataset link and there was a "IMPORT IN PYTHON" button so I clicked on it and there was a snippet for importing our dataset in our code which you can see it below.

as you can see in the last part of this snippet I added target column to our dataset and I used "total_UPDRS" as our targets which seems is a better feature after I checked the paper.

In [2]:
  
parkinson_telemonitoring = fetch_ucirepo(id=189) 
  
X = parkinson_telemonitoring.data.features 
y = parkinson_telemonitoring.data.targets 
  
df = pd.DataFrame(X)
df['total_UPDRS']= y['total_UPDRS']

## classify
after that I had to classify our targets into 2 classes 0 and 1 (making them binary)

so I implemented these functions here so we can use them and check the best result for ourself.

#### kmeans
The K-means clustering algorithm is a method used in data analysis to partition a dataset into K distinct, non-overlapping subgroups (clusters), where each data point belongs to the cluster with the nearest mean.

so we are using this method with K=2.

In [3]:
def Kmeans_classify():
    X_clustering = df.drop('total_UPDRS', axis=1)
    kmeans = KMeans(n_clusters=2, random_state=42).fit(X_clustering)
    df['target'] = kmeans.labels_
    counts = df['target'].value_counts()
    print(counts)

def simple_classify(type):
    threshold = df['total_UPDRS'].median() if(type=='median') else df['total_UPDRS'].mean()
    df['target'] = (df['total_UPDRS'] >= threshold).astype(int)
    counts = df['target'].value_counts()
    print(counts)



## choose method here!
so I added all methods here and you have to uncomment one of them and comment all other ones and it will classify our target with that method.

In [4]:
Kmeans_classify()
simple_classify('median')
simple_classify('mean')

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


target
1    2990
0    2885
Name: count, dtype: int64
target
1    2940
0    2935
Name: count, dtype: int64
target
0    3210
1    2665
Name: count, dtype: int64
